# VER GPU

In [ ]:
!nvidia-smi
import torch

if torch.cuda.is_available():
    print("GPU disponible")
else:
    print("GPU no disponible")

# IMPORTAR JSON

In [ ]:
from custom_transforms import *
from dataset import *
from generarModelo import *
from torch.nn.functional import normalize
from validar_transformaciones import *


izquierda_dir = r"C:/Users/Alejandro/TFG_ENTRENAMIENTO/JSON_FORMADOS/IZQUIERDAO"
derecha_dir = r"C:/Users/Alejandro/TFG_ENTRENAMIENTO/JSON_FORMADOS/DERECHAO"

porcentaje_entrenamiento = 0.6
porcentaje_validacion = 0.2
porcentaje_prueba = 0.2

datos_entrenamiento, datos_validacion, datos_prueba, etiquetas_entrenamiento, etiquetas_validacion, etiquetas_prueba = cargar_y_preparar_datos(izquierda_dir, derecha_dir, porcentaje_entrenamiento, porcentaje_validacion, porcentaje_prueba)

Loading data:  52%|█████▏    | 3985/7694 [00:46<00:43, 86.18it/s] 

# Distribucion

In [ ]:
# Calcular la distribución de clases en los datos de entrenamiento y prueba
def calcular_distribucion_clases(etiquetas):
    clases, recuento = np.unique(etiquetas, return_counts=True)
    distribucion = dict(zip(clases, recuento))
    total_muestras = len(etiquetas)
    for clase, count in distribucion.items():
        porcentaje = (count / total_muestras) * 100
        print(f"Clase {clase}: {count} muestras ({porcentaje:.2f}%)")

# Calcular distribución de clases en los datos de entrenamiento
print("Distribución de clases en datos de entrenamiento:")
calcular_distribucion_clases(etiquetas_entrenamiento)

# Calcular distribución de clases en los datos de prueba
print("\nDistribución de clases en datos de prueba:")
calcular_distribucion_clases(etiquetas_validacion)

print(f"Número de muestras en datos de entrenamiento: {len(datos_entrenamiento)}")
print(f"Número de muestras en datos de prueba: {len(datos_validacion)}")
print(datos_entrenamiento[1].shape)
print("Forma de los datos de entrenamiento:", np.array(datos_entrenamiento).shape)
print("Forma de los datos de prueba:", np.array(datos_validacion).shape)

# COMPARAR ANTES Y DESPUÉS TRANSFORMACION

In [ ]:
#Generar transformacion TAMAÑO
datos_entrenamiento_transformados_zoom = transformacion_zoom(datos_entrenamiento[55],etiquetas_entrenamiento[55])[0]
etiquetas_entrenamiento_transformados_zoom = etiquetas_entrenamiento

#Generar transformacion TRANSLACION EJE X
datos_entrenamiento_transformados_translacionX = desplazar_posesX(datos_entrenamiento[55],etiquetas_entrenamiento[55])[0]
etiquetas_entrenamiento_transformados_translacionX = etiquetas_entrenamiento
#
##Generar transformacion TRANSLACION EJE Y
datos_entrenamiento_transformados_translacionY = desplazar_posesY(datos_entrenamiento[55],etiquetas_entrenamiento[55])[0]
etiquetas_entrenamiento_transformados_translacionY = etiquetas_entrenamiento
#
##Generar transformacion FLIP
datos_entrenamiento_transformados_flip = flip_poses(datos_entrenamiento[55],etiquetas_entrenamiento[55])[0]
etiquetas_entrenamiento_transformados_flip = invertir_etiquetas(etiquetas_entrenamiento)


# VALIDAR TRANSFORMACIONES

In [ ]:
# ZOOM
validacionZoom(datos_entrenamiento,etiquetas_entrenamiento)

In [ ]:
# TRANSLACION X
validacionX(datos_entrenamiento,etiquetas_entrenamiento)

In [ ]:
# TRANSLACION Y
validacionX(datos_entrenamiento,etiquetas_entrenamiento)

In [ ]:
# FLIP
validacionFlip(datos_entrenamiento,etiquetas_entrenamiento)

# VER TENSORES TRANSFORMADOS

In [ ]:
from gifComparaTransformado import *
from generar_gif import gif
from IPython.display import Image


unir_gifs(gif(datos_entrenamiento[55]), gif(datos_entrenamiento_transformados_zoom), "resultadoZoom.gif")

Image("resultadoZoom.gif")


In [ ]:
print(datos_entrenamiento[5])
print((datos_entrenamiento[5])-(datos_entrenamiento_transformados_zoom))

In [ ]:
from IPython.display import Image
Image(gif(datos_entrenamiento[78]))

In [ ]:
from gifComparaTransformado import *
from generar_gif import gif
from IPython.display import Image

unir_gifs(gif(datos_entrenamiento[55]), gif(datos_entrenamiento_transformados_translacionX), "resultadoX.gif")

Image("resultadoX.gif")

In [ ]:
from gifComparaTransformado import *
from generar_gif import gif
from IPython.display import Image

unir_gifs(gif(datos_entrenamiento[55]), gif(datos_entrenamiento_transformados_translacionY), "resultadoY.gif")

Image("resultadoY.gif")

In [ ]:
from gifComparaTransformado import *
from generar_gif import *

from IPython.display import Image


unir_gifs(gif(datos_entrenamiento[55]), gif(datos_entrenamiento_transformados_flip), "resultadoFlip.gif")

Image("resultadoFlip.gif")

# VER LOGS

In [3]:
#%load_ext tensorboard
%reload_ext tensorboard
%tensorboard --logdir runs --port 1291

In [ ]:
#%load_ext tensorboard
%reload_ext tensorboard
%tensorboard --logdir logs/optuna --port 1216

# ENTRENAR OPTUNA

In [12]:
print(datos_entrenamiento.shape)

torch.Size([824, 100, 2, 75])


In [2]:
# Luego, puedes utilizar los DataLoader en las funciones train_with_optuna y train_final_model
from CustomDataset import *

hiperparametros_ranges = {
    'learning_rate': [1e-6, 1e-4],
    'lstm_units': [480, 640, 720, 1080],
    'dense_units': [480, 720, 1080, 1440],
    'dropout_rate_1': [0.15, 0.2, 0.25, 0.3, 0.4, 0.6, 0.8],
    'dropout_rate_2': [0.15, 0.2, 0.25, 0.3, 0.4, 0.6, 0.8],
    'num_lstm_layers': [1,2],
    'kernel_regularizer': [1e-7, 1e-4],
    'conv2d_out_channels': [8, 13, 32], 
    'conv2d_kernel_size': [1, 3, 5], 
    'attention_num_heads': [1, 3, 5],
}


best_model, best_val_loss, best_val_accuracy, best_params = optimize_hyperparameters(datos_entrenamiento, etiquetas_entrenamiento, 
                                                                                     datos_validacion, etiquetas_validacion, 
                                                                                     hiperparametros_ranges, num_trials=300, num_epochs=1000, 
                                                                                     batch_size=16, patience=10)
# Luego, entrenamos el modelo final con los mejores parámetros encontrados por Optuna
#0.69311701

[I 2024-05-08 11:35:35,048] A new study created in memory with name: no-name-d6f78eff-2dcb-44d0-ad37-72d22653478c
Training:   1%|▏         | 13/1000 [03:37<4:35:01, 16.72s/epoch, Best Val Accuracy=0.61636791, Best Val Loss=0.69310840]
[I 2024-05-08 11:39:14,095] Trial 0 finished with value: 0.6163679106670584 and parameters: {'learning_rate': 8.984693596006948e-05, 'lstm_units': 480, 'dense_units': 1080, 'dropout_rate_1': 0.6, 'dropout_rate_2': 0.3, 'num_lstm_layers': 1, 'kernel_regularizer': 3.1451439903162495e-05, 'conv2d_out_channels': 256, 'conv2d_kernel_size': 1, 'attention_num_heads': 8}. Best is trial 0 with value: 0.6163679106670584.


Early stopping at epoch 14 as there's no improvement in validation loss.


Training:   1%|          | 7/1000 [05:07<12:06:45, 43.91s/epoch, Best Val Accuracy=0.50000000, Best Val Loss=0.69323518]
[W 2024-05-08 11:44:24,024] Trial 1 failed with parameters: {'learning_rate': 0.0004271734795374735, 'lstm_units': 1440, 'dense_units': 1080, 'dropout_rate_1': 0.3, 'dropout_rate_2': 0.25, 'num_lstm_layers': 1, 'kernel_regularizer': 4.789964347060574e-05, 'conv2d_out_channels': 128, 'conv2d_kernel_size': 3, 'attention_num_heads': 16} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\Users\Alejandro\anaconda3\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\Alejandro\TFG_ENTRENAMIENTO\PROYECTO_FINAL\generarModelo.py", line 207, in objective
    _, _, val_accuracy = train_model(model, datos_entrenamiento, etiquetas_entrenamiento, hiperparametros, num_epochs, batch_size, patience, trial_number=trial.number)
  File "C:\Users\Alejandro\TFG_ENTRENAMIENTO

KeyboardInterrupt: 

# ENTRENAR FINAL

In [ ]:
#print(study.best_trial.params)

study={'learning_rate': 1.0604608413193053e-05, 
 'lstm_units': 128, 
 'dense_units': 512, 
 'dropout_rate_1': 0.3, 
 'dropout_rate_2': 0.4, 
 'num_lstm_layers': 2, 
 'kernel_regularizer': 0.0030}

In [ ]:
print(study.best_trial.params)

In [ ]:
# Concatenar datos de entrenamiento y validación
datos_final= torch.cat((datos_entrenamiento, datos_validacion), dim=0)

# Concatenar etiquetas de entrenamiento y validación
etiquetas_final = torch.cat((etiquetas_entrenamiento, etiquetas_validacion), dim=0)

modelo_final = train_final_model(datos_final, etiquetas_final, datos_prueba, etiquetas_prueba, study, num_epochs=5000, batch_size=64)
